# Importing libraries

In [17]:
import numpy as np
from astropy.io import fits
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt
from pypower import CatalogMesh, MeshFFTPower, CatalogFFTPower, PowerSpectrumStatistics, utils, setup_logging,CatalogSmoothWindow

# Estimating the survey window function

We will measure the survey window function using the $\texttt{pypower}$ module. We will need to load a pre-measured power spectrum and the randoms position in cartesian coordinate:

In [12]:
def GalacticCapSplit(data, cosmo):
    # Convert to Galactic coordinates using astropy
    coords = SkyCoord(ra=data['RA']*u.degree, dec=data['DEC']*u.degree, frame='icrs')
    galactic_coords = coords.galactic
    
    # Split the data into NGC and SGC using Galactic latitude (b)
    ngc_mask = galactic_coords.b.deg > 0  # NGC: b > 0
    sgc_mask = galactic_coords.b.deg < 0  # SGC: b < 0

    return data[ngc_mask], data[sgc_mask]

def sky_to_cartesian(data, cosmo):
    ra = data['RA']
    dec = data['DEC']
    z = data['Z']
    
    # Convert to Cartesian coordinates
    ra_rad = np.deg2rad(ra)
    dec_rad = np.deg2rad(dec)
    comoving_distance = cosmo.comoving_distance(z).value  # in Mpc
    
    x = comoving_distance * np.cos(dec_rad) * np.cos(ra_rad)
    y = comoving_distance * np.cos(dec_rad) * np.sin(ra_rad)
    z = comoving_distance * np.sin(dec_rad)
    
    return [x,y,z]

def ReadFits(fn):
    with fits.open(fn) as hdul:
        # Assuming the data is in the first extension (index 1)
        data = hdul[1].data
    return data

def load_ps(fn):
    with open(fn, 'r') as file:
        lines = file.readlines()[25:]
    data = np.genfromtxt(lines, dtype = complex)
    data = data.real
    return data.T

In [3]:
# Path to your FITS file
file_path = 'data/DESI/QSO/catalogues/'
fn_random = 'QSO_ffa_NGC_0_clustering.ran.fits'

# Specify the cosmology. Using the DESI cosmology
cosmo = FlatLambdaCDM(H0=67.36, Om0=0.3137721026737606)

In [8]:
#Getting the position of the random catalog in cartesian coordinates
randoms = ReadFits(file_path+fn_random)
randoms_NGC,_ = GalacticCapSplit(randoms, cosmo)
randoms_NGC_positions = sky_to_cartesian(randoms_NGC,cosmo)
randoms_NGC_weights = randoms_NGC['Weight']

In [15]:
#loading the power spectrum
poles = CatalogFFTPower.load('/Users/s2223060/Desktop/PrimordialFeatures/data/DESI/QSO/pk/SecondGenMocks_EZmock_v1_QSO_NGC_0.8-2.1_mock_1_d0.001.npy')

In [ ]:
setup_logging()
# Let us compute the window function multipoles in k-space
boxsize = 12000.
edges = {'step': 2. * np.pi / boxsize}
window_large = CatalogSmoothWindow(randoms_positions1=randoms_NGC_positions, randoms_weights1=randoms_NGC_weights,
                                   power_ref=poles, edges=edges, boxsize=boxsize, position_type='xyz', dtype='f4').poles

[000002.93]  09-17 13:15  CatalogSmoothWindow          INFO     Painting catalog 1 to mesh CatalogMesh(nmesh=[1668 1668 1668], boxsize=[12000. 12000. 12000.], boxcenter=[-1485.90353544  -204.06970576  1480.59355695], dtype=float32).
[000007.78]  09-17 13:15  CatalogMesh                  INFO     Slab 0 ~ 4194304 / 11612435.
[000011.57]  09-17 13:15  CatalogMesh                  INFO     Painted 4194304 out of 11612435 objects to mesh.
[000011.57]  09-17 13:15  CatalogMesh                  INFO     Slab 4194304 ~ 8388608 / 11612435.
[000014.47]  09-17 13:15  CatalogMesh                  INFO     Painted 8388608 out of 11612435 objects to mesh.
[000014.47]  09-17 13:15  CatalogMesh                  INFO     Slab 8388608 ~ 12582912 / 11612435.
[000016.78]  09-17 13:15  CatalogMesh                  INFO     Painted 11612435 out of 11612435 objects to mesh.
[000016.79]  09-17 13:15  CatalogMesh                  INFO     Running interlacing at order 3.
[000520.25]  09-17 13:23  CatalogMesh  